
# Prerequisite

In [ ]:

#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Code, Dataset
from azure.identity import InteractiveBrowserCredential
from azure.ml import dsl
from azure.ml.dsl import Pipeline
from azure.ml.entities import Component

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(credential=InteractiveBrowserCredential(), subscription_id=subscription_id, resource_group_name=resource_group, workspace_name=workspace)

# Pipeline job with registered component
## Register components

In [ ]:
parent_dir = '.'

# please make sure version in the yaml is a new one
train_component = Component.load(path=parent_dir + "/train.yml")
ml_client.components.create_or_update(train_component)


In [ ]:
# please make sure version in the yaml is a new one
score_component = Component.load(path=parent_dir + "/score.yml")
ml_client.components.create_or_update(score_component)

In [ ]:
# please make sure version in the yaml is a new one
eval_component = Component.load(path=parent_dir + "/eval.yml")
ml_client.components.create_or_update(eval_component)

## Build pipeline

In [ ]:
from azure.ml import dsl, MLClient
from azure.ml.dsl import Pipeline
from azure.ml.entities import Dataset
from pathlib import Path


# 1. Get component definition and transfer component to functions
train_component = ml_client.components.get(name="Train", version="0.3")
train_func = dsl.load_component(component=train_component)

score_component = ml_client.components.get(name="score", version="0.3")
score_func = dsl.load_component(component=score_component)

eval_component = ml_client.components.get(name="eval", version="0.3")
eval_func = dsl.load_component(component=eval_component)

# 2. Construct pipeline
@dsl.pipeline(
    compute="cpu-cluster",
    description="E2E dummy train-score-eval pipeline with registered components",
)
def sample_pipeline(
        pipeline_job_training_input,
        pipeline_job_test_input,
        pipeline_job_training_max_epocs,
        pipeline_job_training_learning_rate,
        pipeline_job_learning_rate_schedule,
):
    train_job = train_func(
        training_data=pipeline_job_training_input,
        max_epocs=pipeline_job_training_max_epocs,
        learning_rate=pipeline_job_training_learning_rate,
        learning_rate_schedule=pipeline_job_learning_rate_schedule,
    )
    score_job = score_func(model_input=train_job.outputs.model_output, test_data=pipeline_job_test_input)
    score_job.outputs.score_output.mode = "upload"
    evaluate_job = eval_func(scoring_result=score_job.outputs.score_output)
    return {
        "pipeline_job_trained_model": train_job.outputs.model_output,
        "pipeline_job_scored_data": score_job.outputs.score_output,
        "pipeline_job_evaluation_report": evaluate_job.outputs.eval_output,
    }

# 3. Create pipeline instance

pipeline = sample_pipeline(
    Dataset(local_path=parent_dir + "/data/"),
    Dataset(local_path=parent_dir + "/data/"),
    20,
    1.8,
    "time-based",
)
pipeline.outputs.pipeline_job_trained_model.mode = "upload"
pipeline.outputs.pipeline_job_scored_data.mode = "upload"
pipeline.outputs.pipeline_job_evaluation_report.mode = "upload"

## Submit pipeline job

In [ ]:
# submit job to workspace
ml_client.jobs.create_or_update(pipeline, experiment_name="e2e_registered_components", continue_run_on_step_failure=True)